In [1]:
import pandas as pd
import numpy as np

In [66]:
X5 = np.random.uniform(0,1,size=50).round()

In [2]:
np.random.seed(123)


Y =  np.random.normal(loc=0, scale=15, size=50)
X1 = np.random.normal(loc=10, scale=55, size=50)
X2 = np.random.normal(loc=40, scale=5, size=50)
X3 = np.random.normal(loc=-10, scale=15, size=50)
X4 = np.random.normal(loc=5, scale=25, size=50)

In [67]:
Data = pd.DataFrame({'Y':Y , 'X1':X1, 'X2':X2, 'X3':X3, 'X4':X4, 'X5':X5})

In [92]:
Data['X5'] = Data['X5'].astype('object') # si ponemos astype('category') luego nos salen problemas en el algoritmo creado

In [93]:
Data.dtypes

Y     float64
X1    float64
X2    float64
X3    float64
X4    float64
X5     object
dtype: object

In [4]:
Data.head()

,Y,X1,X2,X3,X4
0,-16.284459,-61.174693,43.210273,-6.192753,22.582753
1,14.960182,-47.133352,30.110560,-5.744120,-9.952633
2,4.244677,105.904172,43.561323,-31.178333,60.017552
3,-22.594421,-33.893450,52.991520,-38.153030,22.207423
4,-8.679004,11.632578,39.876870,-25.294826,4.842319


In [128]:
def regression_tree(Data_set, iterations_vector, x_new):

# POR AHORA SOLO GENERA 3 ITERACIONES EN EL ARBOL --> iterations_vector = range(1,4) como mucho (=[1,2,3])

########################################################################
    
# Data_set tiene que ser tal que, su columna 0 sea Y, y la j-esima sea la variable Xj , para j=1,...,p


    def s_values(j, Data_set):

        s_values = []

        if  (Data.dtypes[j] != 'float') & (Data.dtypes[j] != 'int') : # Para las variables categoricas s_value sera simplemente su rango.

            s_values = Data_set.sort_values(by=[Data_set.columns[j]], axis=0, ascending=True, ignore_index=True).iloc[:, j].unique()


        elif (Data.dtypes[j] == 'float') | (Data.dtypes[j] == 'int') :

            Xj_sorted = Data_set.sort_values(by=[Data_set.columns[j]], axis=0, ascending=True, ignore_index=True).iloc[:, j].unique()

        
            for i in range(0, len(Xj_sorted)-1):

                s_values.append( (Xj_sorted[i] + Xj_sorted[i+1] ) / 2  )

    
        return s_values


########################################################################  

   ## ITERACION 1

    if iterations_vector[0] == 1 : # nacimiento del arbol

        def RSS_1(j,s, Data_set):

         # Data_set tiene que ser tal que, su columna 0 sea Y, y la j-esima sea la variable Xj , para j=1,...,p

    ######################################

            y_i_R1 = Data_set.loc[ Data_set.iloc[:, j] < s , 'Y' ] # y_i tal que x_ij < s

            if len( Data_set.loc[ Data_set.iloc[:, j] < s , 'Y' ] ) == 0 :

                y_R1 = 0

            elif len( Data_set.loc[ Data_set.iloc[:, j] < s , 'Y' ] ) != 0 : 

                y_R1 = ( Data_set.loc[ Data_set.iloc[:, j] < s , 'Y' ]  ).sum() / len( Data_set.loc[ Data_set.iloc[:, j] < s , 'Y' ]  )  
   

            ###############

            y_i_R2 = Data_set.loc[ Data_set.iloc[:, j] >= s , 'Y' ] # y_i tal que x_i1 >= 2


            if len( Data_set.loc[ Data_set.iloc[:, j] >= s , 'Y' ] ) == 0 :

                y_R2 = 0

            elif len( Data_set.loc[ Data_set.iloc[:, j] >= s , 'Y' ] ) != 0 : 

                y_R2 = ( Data_set.loc[ Data_set.iloc[:, j] >= s , 'Y' ]  ).sum() / len( Data_set.loc[ Data_set.iloc[:, j] >= s , 'Y' ]  )  

             ##############

            RSS_11 = ( (y_i_R1 - y_R1 )**2 ).sum()

            RSS_12 = ( (y_i_R2 - y_R2 )**2 ).sum()

            RSS_1 = RSS_11 + RSS_12

            ############

            return(RSS_1)

      ################################################

        ## Busqueda de j_star y s_star de la iteracion 1

        RSS_vector = []
        j_vector = []
        s_vector = []

        s_star_vector = []
        j_star_vector = []
        RSS_star_vector = []

        for j in range(1, Data_set.shape[1]) :

            for s in s_values(j, Data_set) :

                RSS_vector.append( RSS_1(j, s, Data_set) )

                j_vector.append( j )

                s_vector.append( s )

        RSS_df = pd.DataFrame({'RSS':RSS_vector, 'j':j_vector, 's':s_vector})

        RSS_df_sorted = RSS_df.sort_values(by=['RSS'], axis=0, ascending=True, ignore_index=True)

        s_star_vector.append( RSS_df_sorted.loc[0, 's'] )
        j_star_vector.append( RSS_df_sorted.loc[0, 'j'] )
        RSS_star_vector.append(RSS_df_sorted.loc[0, 'RSS'])

        # OJO: s_star_vector[i] sera el s_star de la iteracion i+1 , para i=0,1,...
        # OJO: j_star_vector[i] sera el j_star de la iteracion i+1 , para i=0,1,...    
    
    ######################################################################################

    ## ITERACION 2

    if iterations_vector[1] == 2 :  # Desarrollar nodo R1 de la 1ª iteracion

        ################################################################

        def RSS_2(j,s, Data_set):    # Desarrollar nodo R1 de la 1ª iteracion --> considerar j_star_vector[0] y s_star_vector[0] (1ª iteracion) y < (R1)

         # Data_set tiene que ser tal que, su columna 0 sea Y, y la j-esima sea la variable Xj , para j=1,...,p

           ################

           # Verificando si se cumplen las siguientes dos condiciones conjuntamente nos garantizamos que todas las ramas tienes observaciones de train. 
           # Es decir, no habra ninguna rama sin observaciones de train que caigan en ella.

            cond1 = ( len( Data_set.loc[ (Data_set.iloc[:, j] < s) & (Data_set.iloc[:, j_star_vector[0]] < s_star_vector[0]) , 'Y' ] ) != 0 )
            cond2 = ( len( Data_set.loc[ (Data_set.iloc[:, j] >= s) & (Data_set.iloc[:, j_star_vector[0]] < s_star_vector[0]) , 'Y' ] ) != 0 )
              

            if (cond1 == False) | (cond2 == False) :

                RSS_2 = math.inf

            elif  (cond1 == True) & (cond2 == True) : 

                y_i_R1 = Data_set.loc[ (Data_set.iloc[:, j] < s) & (Data_set.iloc[:, j_star_vector[0]] < s_star_vector[0]) , 'Y' ]

                y_R1 = ( Data_set.loc[ (Data_set.iloc[:, j] < s) & (Data_set.iloc[:, j_star_vector[0]] < s_star_vector[0]) , 'Y' ]  ).sum() / len( Data_set.loc[ (Data_set.iloc[:, j] < s) & (Data_set.iloc[:, j_star_vector[0]] < s_star_vector[0]) , 'Y' ]  )  
   
                
                y_i_R2 = Data_set.loc[ (Data_set.iloc[:, j] >= s) & (Data_set.iloc[:, j_star_vector[0]] < s_star_vector[0]) , 'Y' ]  

                y_R2 = ( Data_set.loc[ (Data_set.iloc[:, j] >= s) & (Data_set.iloc[:, j_star_vector[0]] < s_star_vector[0]) , 'Y' ]  ).sum() / len( Data_set.loc[ (Data_set.iloc[:, j] >= s) & (Data_set.iloc[:, j_star_vector[0]] < s_star_vector[0]) , 'Y' ]  )
           
                
                RSS_21 = ( (y_i_R1 - y_R1 )**2 ).sum()

                RSS_22 = ( (y_i_R2 - y_R2 )**2 ).sum()

                RSS_2 = RSS_21 + RSS_22

            ############

            return(RSS_2)


      ###########################################

      # Busqueda de j_star y s_star de la iteracion 2:

        RSS_vector = []
        j_vector = []
        s_vector = []


        for j in range(1, Data_set.shape[1]) :

            for s in s_values(j, Data_set) :

                RSS_vector.append( RSS_2(j, s, Data_set) )

                j_vector.append( j )

                s_vector.append( s )

        RSS_df = pd.DataFrame({'RSS':RSS_vector, 'j':j_vector, 's':s_vector})

        RSS_df_sorted = RSS_df.sort_values(by=['RSS'], axis=0, ascending=True, ignore_index=True)

        s_star_vector.append( RSS_df_sorted.loc[0, 's'] )
        j_star_vector.append( RSS_df_sorted.loc[0, 'j'] )
        RSS_star_vector.append(RSS_df_sorted.loc[0, 'RSS'])

        # OJO: s_star_vector[i] sera el s_star de la iteracion i+1 , para i=0,1,...
        # OJO: j_star_vector[i] sera el j_star de la iteracion i+1 , para i=0,1,...


####################################################################################

## ITERACION 3

    if iterations_vector[2] == 3 :  # Desarrollar nodo R2 de la 1ª iteracion -->  considerar j_star_vector[0] y s_star_vector[0] (1ª iteracion) y >= (R2)

       #########################################

        def RSS_3(j,s, Data_set):    

         # Data_set tiene que ser tal que, su columna 0 sea Y, y la j-esima sea la variable Xj , para j=1,...,p

           ################

           # Verificando si se cumplen las siguientes dos condiciones conjuntamente nos garantizamos que todas las ramas tienes observaciones de train. 
           # Es decir, no habra ninguna rama sin observaciones de train que caigan en ella.

            cond1 = ( len( Data_set.loc[ (Data_set.iloc[:, j] < s) & (Data_set.iloc[:, j_star_vector[0]] >= s_star_vector[0]) , 'Y' ] ) != 0 )
            cond2 = ( len( Data_set.loc[ (Data_set.iloc[:, j] >= s) & (Data_set.iloc[:, j_star_vector[0]] >= s_star_vector[0]) , 'Y' ] ) != 0 )
              

            if (cond1 == False) | (cond2 == False) :

                RSS_3 = math.inf

            elif  (cond1 == True) & (cond2 == True) : 

                y_i_R1 = Data_set.loc[ (Data_set.iloc[:, j] < s) & (Data_set.iloc[:, j_star_vector[0]] >= s_star_vector[0]) , 'Y' ]

                y_R1 = ( Data_set.loc[ (Data_set.iloc[:, j] < s) & (Data_set.iloc[:, j_star_vector[0]] >= s_star_vector[0]) , 'Y' ]  ).sum() / len( Data_set.loc[ (Data_set.iloc[:, j] < s) & (Data_set.iloc[:, j_star_vector[0]] >= s_star_vector[0]) , 'Y' ]  )  
   
                
                y_i_R2 = Data_set.loc[ (Data_set.iloc[:, j] >= s) & (Data_set.iloc[:, j_star_vector[0]] >= s_star_vector[0]) , 'Y' ]  

                y_R2 = ( Data_set.loc[ (Data_set.iloc[:, j] >= s) & (Data_set.iloc[:, j_star_vector[0]] >= s_star_vector[0]) , 'Y' ]  ).sum() / len( Data_set.loc[ (Data_set.iloc[:, j] >= s) & (Data_set.iloc[:, j_star_vector[0]] >= s_star_vector[0]) , 'Y' ]  )
           
                
                RSS_31 = ( (y_i_R1 - y_R1 )**2 ).sum()

                RSS_32 = ( (y_i_R2 - y_R2 )**2 ).sum()

                RSS_3 = RSS_31 + RSS_32
            

            ############

            return(RSS_3)


       ###########################################

      # Busqueda de j_star y s_star de la iteracion 3:

        RSS_vector = []
        j_vector = []
        s_vector = []


        for j in range(1, Data_set.shape[1]) :

            for s in s_values(j, Data_set) :

                RSS_vector.append( RSS_3(j, s, Data_set) )

                j_vector.append( j )

                s_vector.append( s )

        RSS_df = pd.DataFrame({'RSS':RSS_vector, 'j':j_vector, 's':s_vector})

        RSS_df_sorted = RSS_df.sort_values(by=['RSS'], axis=0, ascending=True, ignore_index=True)

        s_star_vector.append( RSS_df_sorted.loc[0, 's'] )
        j_star_vector.append( RSS_df_sorted.loc[0, 'j'] )
        RSS_star_vector.append(RSS_df_sorted.loc[0, 'RSS'])

        # OJO: s_star_vector[i] sera el s_star de la iteracion i+1 , para i=0,1,...
        # OJO: j_star_vector[i] sera el j_star de la iteracion i+1 , para i=0,1,...

    ###############################################

    ## Nº de observaciones en cada rama del arbol generado

    obs_rama_1 = len( Data_set.loc[ (Data_set.iloc[:, j_star_vector[0] ] < s_star_vector[0])  &  (Data_set.iloc[:, j_star_vector[1] ] < s_star_vector[1]) , : ] )
    obs_rama_2 = len( Data_set.loc[ (Data_set.iloc[:, j_star_vector[0] ] < s_star_vector[0])  &  (Data_set.iloc[:, j_star_vector[1] ] >= s_star_vector[1]) , :])
    obs_rama_3 = len( Data_set.loc[ (Data_set.iloc[:, j_star_vector[0] ] >= s_star_vector[0]) &  (Data_set.iloc[:, j_star_vector[2] ] < s_star_vector[2]) , :])
    obs_rama_4 = len( Data_set.loc[ (Data_set.iloc[:, j_star_vector[0] ] >= s_star_vector[0])  & (Data_set.iloc[:, j_star_vector[2] ] >= s_star_vector[2]) , :] )

    obs_ramas = [obs_rama_1, obs_rama_2, obs_rama_3, obs_rama_4]

   ###############################################
    
    ## PREDICCIONES


    # si x_new cae en rama 1

    if (x_new[j_star_vector[0] - 1] < s_star_vector[0]) & (x_new[j_star_vector[1] - 1] < s_star_vector[1]):  

        y_new_predict = Data_set.loc[ (Data_set.iloc[:, j_star_vector[0] ] < s_star_vector[0])  &  (Data_set.iloc[:, j_star_vector[1] ] < s_star_vector[1]) , 'Y' ]

        y_new_predict = y_new_predict.sum() / obs_rama_1


    # si x_new cae en rama 2   

    elif (x_new[j_star_vector[0] - 1] < s_star_vector[0]) & (x_new[j_star_vector[1] - 1] >= s_star_vector[1]):  

        y_new_predict = Data_set.loc[ (Data_set.iloc[:, j_star_vector[0] ] < s_star_vector[0])  &  (Data_set.iloc[:, j_star_vector[1] ] >= s_star_vector[1]) , 'Y' ]

        y_new_predict = y_new_predict.sum() / obs_rama_2

    
    # si x_new cae en rama 3

    elif (x_new[j_star_vector[0] - 1] >= s_star_vector[0]) & (x_new[j_star_vector[1] - 1] < s_star_vector[1]):  

        y_new_predict = Data_set.loc[ (Data_set.iloc[:, j_star_vector[0] ] >= s_star_vector[0])  &  (Data_set.iloc[:, j_star_vector[1] ] < s_star_vector[1]) , 'Y' ]

        y_new_predict = y_new_predict.sum() / obs_rama_3

     
     # si x_new cae en rama 4

    elif (x_new[j_star_vector[0] - 1] >= s_star_vector[0]) & (x_new[j_star_vector[1] - 1] >= s_star_vector[1]): 

        y_new_predict = Data_set.loc[ (Data_set.iloc[:, j_star_vector[0] ] >= s_star_vector[0])  &  (Data_set.iloc[:, j_star_vector[1] ] >= s_star_vector[1]) , 'Y' ]

        y_new_predict = y_new_predict.sum() / obs_rama_4


    
    return(j_star_vector, s_star_vector, RSS_star_vector, obs_ramas, y_new_predict ) 


Arbol con 3 iteraciones, tiene 4 ramas:  (con k iter tiene k+1 ramas)

rama 1: Data.iloc[:, j_star_vector[0] ] < s_star_vector[0]  & Data.iloc[:, j_star_vector[1] ] < s_star_vector[1]   

rama 2: Data.iloc[:, j_star_vector[0] ] < s_star_vector[0]  & Data.iloc[:, j_star_vector[1] ] >= s_star_vector[1]   

rama 3: Data.iloc[:, j_star_vector[0] ] >= s_star_vector[0]  & Data.iloc[:, j_star_vector[2] ] < s_star_vector[2]   

rama 4: Data.iloc[:, j_star_vector[0] ] >= s_star_vector[0]  & Data.iloc[:, j_star_vector[2] ] >= s_star_vector[2]   

In [88]:
x1 = np.random.normal(loc=10, scale=55, size=1)
x2 = np.random.normal(loc=40, scale=5, size=1)
x3 = np.random.normal(loc=-10, scale=15, size=1)
x4 = np.random.normal(loc=5, scale=25, size=1)
x5 = np.random.uniform(0,1,size=1).round()

In [89]:
x_new = pd.Series({'X1': x1 , 'X2':x2, 'X3': x3, 'X4': x4, 'X5':x5})

In [90]:
x_new

X1    [13.227809114745043]
X2     [45.32211344205454]
X3    [-10.58994001967373]
X4     [41.22259029064402]
X5                   [0.0]
dtype: object

In [129]:
j_star_vector, s_star_vector, RSS_star_vector, obs_ramas, y_new_predict = regression_tree(Data, range(1,4), x_new)

In [130]:
j_star_vector

[3, 1, 2]

In [131]:
s_star_vector

[-4.0399734732289145, -58.19656745289842, 33.82252104321174]

In [132]:
RSS_star_vector

[14585.364015047191, 8833.262064304377, 2603.9472163368296]

In [133]:
obs_ramas

[4, 37, 1, 8]

In [134]:
y_new_predict

4.226418699578563